<a href="https://colab.research.google.com/github/tylee33/NLP-Lecture/blob/master/%5BNLP%EA%B0%95%EC%9D%98_5%5D_Text_Classification_(DNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Deep Neural Networks

### Deep Neural Networks 아키텍처는 각 단일 레이어가 이전의 연결 만 수신하고 숨겨진 부분의 다음 레이어에만 연결을 제공하는 여러 레이어의 연결을 통해 학습하도록 설계되었습니다. 입력은 피처 공간의 연결입니다 (첫 번째 숨겨진 레이어의 Feature_extraction 섹션에서 논의한 것처럼 DNN (Deep Neural Networks)의 경우 입력 레이어는 그림의 표준 DNN에 표시된 것처럼 tf-ifd, 단어 포함 등이 될 수 있습니다. 출력 레이어에는 멀티 클래스 분류의 클래스 수와 이진 분류의 경우 하나의 뉴런 만있는 뉴런이 있지만이 백서의 주요 기여는 다른 목적을 위해 훈련 된 DNN이 많이 있다는 것입니다. 각 학습 모델이 무작위로 생성되는 곳 (각 계층의 노드 수와 계층 수는 임의로 할당 됨) DNN (Deep Neural Network)의 구현은 기본적으로 표준 역 전파 알고리즘과 시그 모이 드를 사용하는 차별적으로 훈련 된 모델입니다. 활성화 함수로서의 ReLU 멀티 클래스 분류를위한 출력 레이어는 Softmax를 사용해야합니다.

![그림](https://github.com/kk7nc/Text_Classification/raw/master/docs/pic/DNN.png)

### import package

In [1]:
from sklearn.datasets import fetch_20newsgroups
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics

Using TensorFlow backend.


### convert text to TF-IDF:

In [0]:
def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
    vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS)
    X_train = vectorizer_x.fit_transform(X_train).toarray()
    X_test = vectorizer_x.transform(X_test).toarray()
    print("tf-idf with",str(np.array(X_train).shape[1]),"features")
    return (X_train,X_test)

### Build a DNN Model for Text:

In [0]:
def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

### Load text dataset (20newsgroups):

In [0]:
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')
X_train = newsgroups_train.data
X_test = newsgroups_test.data
y_train = newsgroups_train.target
y_test = newsgroups_test.target

### run DNN and see our result:

In [5]:
X_train_tfidf,X_test_tfidf = TFIDF(X_train,X_test)
model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 20)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_DNN.predict(X_test_tfidf)

print(metrics.classification_report(y_test, predicted.argmax(axis=1)))

W0815 23:28:44.593965 140600686536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0815 23:28:44.606236 140600686536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0815 23:28:44.609463 140600686536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0815 23:28:44.625021 140600686536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is deprecated. Please use tf.compat.v1.placeholder_with_default instead.

W0815 23:28:44.632684 

tf-idf with 75000 features


W0815 23:28:44.785562 140600686536576 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3341: The name tf.log is deprecated. Please use tf.math.log instead.

W0815 23:28:44.870919 140600686536576 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 11314 samples, validate on 7532 samples
Epoch 1/10
 - 16s - loss: 2.7924 - acc: 0.0959 - val_loss: 2.1340 - val_acc: 0.2549
Epoch 2/10
 - 13s - loss: 1.3636 - acc: 0.5205 - val_loss: 0.8794 - val_acc: 0.7258
Epoch 3/10
 - 12s - loss: 0.4748 - acc: 0.8418 - val_loss: 0.7429 - val_acc: 0.8002
Epoch 4/10
 - 12s - loss: 0.1915 - acc: 0.9428 - val_loss: 0.8093 - val_acc: 0.8169
Epoch 5/10
 - 13s - loss: 0.1087 - acc: 0.9676 - val_loss: 0.8334 - val_acc: 0.8209
Epoch 6/10
 - 13s - loss: 0.0693 - acc: 0.9768 - val_loss: 0.8756 - val_acc: 0.8172
Epoch 7/10
 - 12s - loss: 0.0505 - acc: 0.9867 - val_loss: 0.9184 - val_acc: 0.8263
Epoch 8/10
 - 12s - loss: 0.0421 - acc: 0.9888 - val_loss: 0.9778 - val_acc: 0.8176
Epoch 9/10
 - 13s - loss: 0.0448 - acc: 0.9877 - val_loss: 0.9164 - val_acc: 0.8229
Epoch 10/10
 - 12s - loss: 0.0359 - acc: 0.9907 - val_loss: 0.9834 - val_acc: 0.8240
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       319
 